## Xenium Spatial DE Test

In [1]:
from Neighborhood_DE import Neighborhood
import numpy as np
import pandas as pd
import scanpy as sc

In [2]:
adata = sc.read_h5ad('data/Xenium_5K_breast_cancer.h5ad')

#### Example 1: Sender Mode

Find sender cells of the types you specified.

Divide them into:

- Group 1: sender cells expressing PDL1.

- Group 2: sender cells not expressing PDL1.

- Extra filter: only keep sender cells where ≥ 50% of their neighbors are of the receiver type (e.g., "Cancer Epithelial").

- Perform DE between Group 1 vs Group 2.

Biological question:
"What genes are differentially expressed between CXCL12-posiive and CXCL12-negative sender cells, considering only senders surrounded by the receiver cell type"

In [3]:
celltype_receiver = {
    'celltype': ['T-cells', 'Myeloid', 'B-cells'],
    'rctd_celltype': ['tcell', 'myeloid', 'bcells']
}

celltype_sender = {
    'celltype': ['Cancer Epithelial'],
    'rctd_celltype': ['epithelial']
}

method = Neighborhood(
    gene='CXCL12',
    celltype_sender=celltype_sender,
    celltype_receiver=celltype_receiver,
    adata=adata,
    sender=True,
    receiver=False,
    neighbor_perc=0.1  # optional: require at least 50% of neighbors be receiver cell type
)

# Run DE test without bootstrapping
diff_expr = method.de_test(
    nperm=0,
    sort=True,
    test='wilcoxon'
)


['Cancer Epithelial'] expressing gene CXCL12 at num cells: 8993
neighborhood cell mask length of [    15     16     17 ... 577255 577256 577257]
fraction above 50% has size 1
['Cancer Epithelial'] expressing gene CXCL12 filtered by neighbor percentage at num cells: 176
group 1 indices of length: 176
group2 indices of length: 13870


/media/qian/vol2/Ivy/envs/xenium-env/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


### Example 2: Sender mode + Neighbor Gene Filter

Group 1: PDCD1+ sender cells.

Group 2: PDCD1- sender cells.

Extra filter: only keep sender cells that have at least one neighbor expressing CD274.

Perform DE between Group 1 vs Group 2.

Biological question:
"What genes distinguish PDL1+ vs PDL1- sender cells, among those surrounded by receiver cells and exposed to CD274-expressing neighbors"

In [3]:
celltype_receiver = {
    'celltype': ['T-cells', 'Myeloid', 'B-cells'],
    'rctd_celltype': ['tcell', 'myeloid', 'bcells']
}

celltype_sender = {
    'celltype': ['Cancer Epithelial'],
    'rctd_celltype': ['epithelial']
}

method = Neighborhood(
    gene='PDCD1',
    neighbor_gene='CD274',
    celltype_sender=celltype_sender,
    celltype_receiver=celltype_receiver,
    adata=adata,
    sender=True,
    receiver=False,
    neighbor_perc=0  # e.g., no neighbor celltype defined, just by neighobr_gene
)

diff_expr = method.de_test(
    nperm=0,
    sort=True
)


['Cancer Epithelial'] expressing gene PDCD1 at num cells: 515
group 1 indices of length: 137
group2 indices of length: 25791


/media/qian/vol2/Ivy/envs/xenium-env/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


### Example 3: Receiver Mode
Focus on receiver cells.
Divide them into:

- Group 1: receiver cells that have at least one neighbor expressing PDL1.
- Group 2: receiver cells without neighbors expressing PDL1.

Biological question:
"What genes are differentially expressed in receiver cells that are near PDL1+ cells compared to those not near PDL1+ cells?"

This looks at cells surrounding gene expression
1. Masks expression by those with gene expression > 0 for {gene} (ex: CXCL12)
2. Determines neighbors of genes from mask in step 1
3. is celltype_receiver is set, then filter out the neighbors by celltype receiver (ex: neighbor cells to CXCL12 +/- that are immune cells)

In [5]:
celltype_receiver = {
    'celltype': ['T-cells', 'Myeloid', 'B-cells'],
    'rctd_celltype': ['tcell', 'myeloid', 'bcells']
}

celltype_sender = {
    'celltype': ['Cancer Epithelial'],
    'rctd_celltype': ['epithelial']
}

method = Neighborhood(
    gene='CXCL12',
    celltype_sender=None,
    celltype_receiver=celltype_receiver,
    adata=adata,
    sender=False,
    receiver=True
)

diff_expr = method.de_test(
    nperm=0,
    sort=True
)


group 1 indices of length: 100911
group2 indices of length: 261913


/media/qian/vol2/Ivy/envs/xenium-env/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


KeyboardInterrupt: 